# Install libraries

# Imports

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import torchvision
import numpy as np
import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 200
from tqdm import tqdm

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Unzip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip "/content/drive/MyDrive/Bachelor's Project/data/original/redfin_images.zip"

# Config

In [ ]:
image_size = 256

# Model Architecture

In [ ]:
class VariationalEncoder(nn.Module):
    def __init__(self, latent_dims, image_size):
        super(VariationalEncoder, self).__init__()
        self.image_size = image_size
        self.linear1 = nn.Linear(self.image_size * self.image_size, 512)
        nn.init.xavier_uniform_(self.linear1.weight)
        self.bn1 = nn.BatchNorm1d(512)  # Add batch normalization
        self.linear2 = nn.Linear(512, latent_dims)
        nn.init.xavier_uniform_(self.linear2.weight)
        self.bn2 = nn.BatchNorm1d(latent_dims)  # Add batch normalization
        self.linear3 = nn.Linear(512, latent_dims)
        nn.init.xavier_uniform_(self.linear3.weight)
        self.bn3 = nn.BatchNorm1d(latent_dims)  # Add batch normalization
        self.N = torch.distributions.Normal(0, 1)
        self.N.loc = self.N.loc.cuda()
        self.N.scale = self.N.scale.cuda()
        self.kl = 0

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.bn1(self.linear1(x)))  # Apply batch normalization
        mu = self.bn2(self.linear2(x))  # Apply batch normalization
        sigma = self.bn3(self.linear3(x))  # Apply batch normalization
        sigma = torch.clamp(sigma, min=1e-2, max=1e2)
        z = mu + sigma * self.N.sample(mu.shape)
        self.kl = (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum()
        return z

In [ ]:
class Decoder(nn.Module):
    def __init__(self, latent_dims, image_size):
        super(Decoder, self).__init__()
        self.image_size = image_size
        self.linear1 = nn.Linear(latent_dims, 512)
        nn.init.xavier_uniform_(self.linear1.weight)
        self.bn1 = nn.BatchNorm1d(512)  # Add batch normalization
        self.linear2 = nn.Linear(512, self.image_size * self.image_size)
        nn.init.xavier_uniform_(self.linear2.weight)
        self.bn2 = nn.BatchNorm1d(self.image_size * self.image_size)  # Add batch normalization

    def forward(self, z):
        z = F.relu(self.bn1(self.linear1(z)))  # Apply batch normalization
        z = torch.sigmoid(self.bn2(self.linear2(z)))  # Apply batch normalization
        return z.reshape((-1, 1, self.image_size, self.image_size))

In [ ]:
class VariationalAutoencoder(nn.Module):
    def __init__(self, latent_dims, image_size):
        super(VariationalAutoencoder, self).__init__()
        self.encoder = VariationalEncoder(latent_dims, image_size=image_size)
        self.decoder = Decoder(latent_dims, image_size=image_size)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

# Plots

In [ ]:
def plot_reconstructed(autoencoder, r0=(-5, 10), r1=(-10, 5), n=12):
    w = 256
    img = np.zeros((n*w, n*w))
    for i, y in enumerate(np.linspace(*r1, n)):
        for j, x in enumerate(np.linspace(*r0, n)):
            z = torch.Tensor([[x, y]]).to(device)
            x_hat = autoencoder.decoder(z)
            x_hat = x_hat.reshape(256, 256).to('cpu').detach().numpy()
            img[(n-1-i)*w:(n-1-i+1)*w, j*w:(j+1)*w] = x_hat
    plt.imshow(img, extent=[*r0, *r1])

In [ ]:
def plot_latent(autoencoder, data, num_batches=100):
    for i, (x, y) in enumerate(data):
        z = autoencoder.encoder(x.to(device))
        z = z.to('cpu').detach().numpy()
        plt.scatter(z[:, 0], z[:, 1], c=y, cmap='tab10')
        if i > num_batches:
            plt.colorbar()
            break

In [ ]:
import matplotlib.pyplot as plt
import torchvision.utils as vutils

def plot_images(original_images, reconstructed_images, device):
    """
    Plots the original and reconstructed images.

    Args:
        original_images (Tensor): Batch of original images.
        reconstructed_images (Tensor): Batch of reconstructed images.
        device (torch.device): Device where the data resides.
    """
    # Move the images to CPU for plotting
    original_images = original_images.cpu()
    reconstructed_images = reconstructed_images.cpu()

    # Create a grid of images
    image_grid = vutils.make_grid(original_images, nrow=8, normalize=True)
    recon_grid = vutils.make_grid(reconstructed_images, nrow=8, normalize=True)

    # Plot the images
    plt.figure(figsize=(16, 8))
    plt.subplot(1, 2, 1)
    plt.imshow(image_grid.permute(1, 2, 0))
    plt.title('Original Images')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(recon_grid.permute(1, 2, 0))
    plt.title('Reconstructed Images')
    plt.axis('off')

    plt.show()

# Training script

In [ ]:
from tqdm.autonotebook import tqdm

In [ ]:
import torch
from torchvision.utils import make_grid

def train(autoencoder, data, epochs=20):
    opt = torch.optim.Adam(autoencoder.parameters())
    scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=100, gamma=0.1)  # Learning rate scheduler
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    autoencoder = autoencoder.to(device)
    loss_val = 0

    for epoch in tqdm(range(epochs)):
        for batch_idx, x in enumerate(tqdm(data, total=len(data))):
            x = x.to(device)
            opt.zero_grad()

            x_hat = autoencoder(x)
            loss = ((x - x_hat)**2).sum() + autoencoder.encoder.kl

            loss.backward()
            opt.step()
            scheduler.step()
            loss_val += loss.item()
            # Visualize original and reconstructed images
            if (batch_idx + 1) % 100 == 0:
                print(f'Epoch [{epoch+1}/{epochs}], Step [{batch_idx+1}/{len(data)}], Loss: {loss_val / 100:.4f}')
                loss_val = 0
                with torch.no_grad():
                    original_images = make_grid(x[:8], nrow=8, normalize=True)
                    reconstructed_images = make_grid(x_hat[:8], nrow=8, normalize=True)

                    # Log or save images
                    # (Implementation depends on your specific use case)
                    plot_images(original_images, reconstructed_images, device)

    return autoencoder

# Dataset

In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.image_files = [f for f in os.listdir(data_dir) if f.endswith('.jpg') or f.endswith('.png')]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.data_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

# wafaw

In [ ]:
image_size=256

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),  # Resize images to 28x28 pixels
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize image tensors
])

# dataset = CustomImageDataset('/content/redfin_images', transform=transform)
dataset = torch.utils.data.DataLoader(
        torchvision.datasets.MNIST('./data',
               transform=torchvision.transforms.ToTensor(),
               download=True),
        batch_size=128,
        shuffle=True)
data_loader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=2)

vae = VariationalAutoencoder(128, image_size=image_size).to(device) # GPU

In [ ]:
vae = train(vae, dataset)